In [1]:
!pip install transformers datasets
!pip install git+https://github.com/huggingface/trl.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
  Cloning https://github.com/huggingface/trl.git to /tmp/pip-req-build-flcszpjb
  Running command git clone --filter=blob:none --quiet https://github.co

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import DPOTrainer, DPOConfig
from datasets import load_dataset, Dataset
import json

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import sys
FOLDERNAME = 'Colab Notebooks/CS 234/cs234'
sys.path.append(f'/content/drive/My Drive/{FOLDERNAME}')


%cd /content/drive/MyDrive/"Colab Notebooks/CS 234/cs234"
! ls # verify that you are in the right directory

/content/drive/MyDrive/Colab Notebooks/CS 234/cs234
alfred	  data_examples  lib	      README.md		setup.py
AutoGPTQ  dockerfile	 LICENSE      requirements.txt	tests
build	  include	 MANIFEST.in  scripts		toolformer
configs   index_files	 nbs	      settings.ini	toolformer.egg-info


In [3]:
# load model and dataset - dataset needs to be in a specific format
model_id = "bigscience/bloom-1b1"
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.13G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [7]:
# get dataset
import pandas as pd

example_df = pd.read_csv("data_examples/Weather.csv")
dpo_dataset = example_df.rename(columns={'input': 'prompt',
                                         "text": "chosen",
                                         "candidate": "rejected"
                                         })
train_dataset = Dataset.from_dict(dpo_dataset)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1536)
    (word_embeddings_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1536, out_features=4608, bias=True)
          (dense): Linear(in_features=1536, out_features=1536, bias=True)
          (attention_dropout): Dropout(p=0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1536, out_features=6144, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=6144, out_features=1536, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
  )
  (lm

In [11]:
# load trainer
NUM_TRAIN_EPOCHS = 10
OUTPUT_DIR = f"output/epoch={NUM_TRAIN_EPOCHS}"
training_args = DPOConfig(
    beta=0.1,
    # does not automatically save model output
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_TRAIN_EPOCHS
)
trainer = DPOTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
)

# train
results = trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:363: UserWarning: `max_length` is not set in the DPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:376: UserWarning: `max_prompt_length` is not set in the DPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:411: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/504 [00:00<?, ? examples/s]

Step,Training Loss
500,0.000000


In [12]:
# save results
with open(f"{OUTPUT_DIR}/dpo_results.json", "w") as f:
    json.dump(results.metrics, f)
model.save_pretrained(OUTPUT_DIR)

In [13]:
dpo_dataset.head()

,prompt,chosen,rejected
0,\r\nCould you provide the current weather cond...,"London is currently facing heavy rain, so it’s...","London is currently facing [WeatherAPI(""London..."
1,"What is the current weather in London, includi...","London is currently facing heavy rain, so it’s...","London is [WeatherAPI(""London"")]→overcast clou..."
2,What's the current weather in London and the e...,"London is currently facing heavy rain, so it’s...","London is currently [WeatherAPI(""London"")]→ove..."
3,"What is the current weather in Berlin, includi...","The skies over Berlin are clear today, making ...","The skies over Berlin are [WeatherAPI(""Berlin""..."
4,What is the current weather in Tokyo?,"In Tokyo, it is currently snowing, which is gr...","In Tokyo, it is currently [WeatherAPI(""Tokyo"")..."
